<a href="https://colab.research.google.com/github/Matheusfarmaceutico/Estat-stica-e-An-lise-de-dados-com-Python/blob/main/01_Preparando_os_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 44

In [ ]:
db = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/01 - Preparação dos dados/02 - Meus arquivos/01 - Preparando o DB/microdados_enem_2019_sp.csv", sep  = ";", encoding = "iso-8859-1")

# **Extração e Limpeza de Dados**

---



---



In [ ]:

db.head()

In [ ]:
db.shape

In [ ]:
db.dtypes #complex: raiz quadrada de -9, exemplo.

In [ ]:
dados01 = db.drop(columns = ["CO_MUNICIPIO_RESIDENCIA"])

In [ ]:
dados01.head()

In [ ]:
dados01 = dados01.drop(columns = ["CO_UF_RESIDENCIA", "SG_UF_RESIDENCIA","CO_MUNICIPIO_NASCIMENTO"],)

In [ ]:
dados01.head(40)

In [ ]:
dados1 = dados01.drop(columns = ["NO_MUNICIPIO_NASCIMENTO","CO_UF_NASCIMENTO","SG_UF_NASCIMENTO","TP_ANO_CONCLUIU","TP_ENSINO","CO_MUNICIPIO_ESC","CO_UF_ESC","SG_UF_ESC"],)

In [ ]:
dados1.head()

In [ ]:
dados01 = dados1.drop(columns = ["TP_DEPENDENCIA_ADM_ESC","TP_LOCALIZACAO_ESC",'TP_SIT_FUNC_ESC'])

In [ ]:
dados01.head()
dados01.shape

# **Corrigir erros nas notas das disciplinas**

In [ ]:
dados01.loc[:,"NU_NOTA_MT"] /= 10 #O dois pontos do começo significam que a div
#acontecerá em todas as linhas.
dados01.loc[:,"NU_NOTA_CH"] /= 10
dados01.loc[:,"NU_NOTA_LC"] /= 10
dados01.loc[:,"NU_NOTA_CN"] /= 10



In [ ]:
dados01.head()

In [ ]:
dados01 = dados01.rename(columns ={"NU_NOTA_CN":"NOTA_CN","NU_NOTA_CH":"NOTA_CH",
"NU_NOTA_LC":"NOTA_LC","NU_NOTA_MT":"NOTA_MT","NU_NOTA_REDACAO":"NOTA_REDACAO"})

In [ ]:
dados01 = dados01.rename(columns= {"NU_COMP1":"COMP1",
                         "NU_NOTA_COMP2":"COMP2",
                         "NU_NOTA_COMP3":"COMP3",
                         "NU_NOTA_COMP4":"COMP4",
                         "NU_NOTA_COMP5":"COMP5"})

In [ ]:
dados01.head()

In [ ]:
dados01 = dados01.rename(columns = {"NU_IDADE":"IDADE",
                                    "TP_SEXO":"SEXO",
                                    "TP_COR_RACA":"RACA",
                                    "Q025":"INTERNET",
                                    "TP_ESCOLA":"ESCOLA"})

In [ ]:
dados01.head()

In [ ]:
dados01["RACA"] = dados01["RACA"].replace({0:"nao_Declarado",
                                           1:"branca",
                                           2:"preta",
                                           3:"parda",
                                           4:"amarela",
                                           5:"indigena"})

In [ ]:
dados01.head()

In [ ]:
dados01["TP_LINGUA"] = dados01["TP_LINGUA"].replace([0,1], ["Inglês","Espanhol"]) #outra maneira de fazer o replace

In [ ]:
dados01.head()

In [ ]:
dados01["ESCOLA"] = dados01["ESCOLA"].replace({1:"nao_respondeu",
                                               2:"publica",
                                               3:"privada",
                                               4:"exterior"})

In [ ]:
dados01.head()

In [ ]:
dados01["INTERNET"] = dados01["INTERNET"].replace({"A":"sim","B":"nao"})

In [ ]:
dados01.head()

**Análise de Idades**

In [ ]:
dados01["IDADE"].value_counts()

In [ ]:
dados01["IDADE"].value_counts().sort_index(ascending=True) #.sort index coloca as idades em ordem crescente 
# quando n leva um argumento (por padrão), mas é possível setar para que o ascending=False faça com que os
#dados apareçam em modo decrescente.

**É notável que haja no db alunos menores de 12 anos, o que pode indicar erros nessa tabela. Dessa forma, vamos filtrar a tabela para que haja somente idades válidas (Idades maiores do que 11 anos)**


In [ ]:
# Vamos verificar primeiro o número de alunos menores do que 12 anos por município
menores_12 = dados01.query("IDADE < 12")["NO_MUNICIPIO_RESIDENCIA"].value_counts()
menores_12

**Aqui vamos, de fato, selecionar apenas aqueles maiores do que 11 anos**

In [ ]:
maiores_11 = dados01.loc[dados01.IDADE > 11]

In [ ]:
maiores_11["IDADE"].value_counts().sort_index()

**Lidando com treineiros**

In [ ]:
# 1 primeiramente vamos checar quantos treineiros estão em nossa base de dados.
maiores_11.query("IN_TREINEIRO == 1")["IN_TREINEIRO"].value_counts()

In [ ]:
# 2 vamos criar uma tabela somente com treineiros, para quem sabe podermos trabalhar com eles futuramente.
treineiros = maiores_11.loc[maiores_11.IN_TREINEIRO == 1]


In [ ]:
#3 Vamos visualizar quantos desses alunos eram treineiros (saída = 93.988)
treineiros.query("IN_TREINEIRO == 1").shape


In [ ]:
#4 Vamos verificar quantos alunos Vestibulandos (não treineiros) estão em nossa base de dados.
#(saída = 719.768)
maiores_11.query("IN_TREINEIRO == 0").shape

In [ ]:
# 5 - Por fim, vamos salvar o DB Treineiros como um arquivo Csv, para eventualmente utilizá-lo.
#treineiros.to_csv("treineiros_enem_sp_2019.csv",encoding = "iso-8859-1",index = False)

#OBS: ESTÁ COMO COMENTÁRIO PARA QUE A CÉDULA N FIQUE GERANDO UM NOVO ARQUIVO A CADA VEZ QUE FOR EXECUTADA.

**Lidando com Vestibulandos**

In [ ]:
#Vamos separar os vestibulandos dos treineiros

vestibulandos = maiores_11.loc[maiores_11.IN_TREINEIRO == 0]

In [ ]:
#Contabilizando os  719.768 vestibulandos
vestibulandos.shape

**Análise de presenças**


* Pessoas que estiveram presentes (Identificados pelo número 1)
* Faltosos (Identificados pelo número 0)
* Eliminados do Enem (Identificados pelo número 2)




In [ ]:
vestibulandos["TP_PRESENCA_CN"].value_counts()

In [ ]:
vestibulandos["TP_PRESENCA_CH"].value_counts()

In [ ]:
vestibulandos["TP_PRESENCA_LC"].value_counts() #LC E CH no mesmo dia. MT E CN no mesmo dia.

In [ ]:
vestibulandos["TP_PRESENCA_MT"].value_counts()

RESUMO DA PRESENÇA NAS PROVAS:

Ciências da Natureza: 500805 presentes, 218693 faltaram e 270 eliminados.

Ciências Humanas: 533142 presentes, 185969 faltaram e 657 eliminados.

Linguagens e Códigos: 533142 presentes, 185969 faltaram e 657 eliminados.

Matemática: 500805 presentes, 218693 faltaram e 270 eliminados.

**Redação**
1 Sem problemas
2 Anulada
3 Cópia Texto Motivador
4 Em Branco
6 Fuga ao tema
7 Não atendimento ao tipo textual
8 Texto insuficiente
9 Parte desconectada

In [ ]:
vestibulandos["TP_STATUS_REDACAO"].value_counts().sort_index()

**Valores Ausentes**

In [ ]:
vestibulandos["IDADE"].isnull().sum() #0 valores nulos 

In [ ]:
provas = ["NOTA_REDACAO","NOTA_CN","NOTA_CH","NOTA_LC","NOTA_MT"]

In [ ]:
vestibulandos[provas].isnull().sum()

In [114]:
presenca_total =  ["TP_PRESENCA_CN",
                   "TP_PRESENCA_CH",
                    "TP_PRESENCA_LC",
                    "TP_PRESENCA_MT"
]

In [117]:
vestibulandos["presenca_total"] = vestibulandos[presenca_total].sum(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [118]:
vestibulandos.head()

,NU_INSCRICAO,NO_MUNICIPIO_RESIDENCIA,IDADE,SEXO,TP_ESTADO_CIVIL,RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,ESCOLA,IN_TREINEIRO,CO_ESCOLA,NO_MUNICIPIO_ESC,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NOTA_CN,NOTA_CH,NOTA_LC,NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,COMP2,COMP3,COMP4,COMP5,NOTA_REDACAO,INTERNET,presenca_total
0,"1,90E+11",Santa Isabel,22,M,1,parda,1,1,nao_respondeu,0,NaN,NaN,1,1,1,1,564.6,585.8,592.9,600.2,Inglês,1.0,160.0,200.0,180.0,200.0,200.0,940.0,nao,4
1,"1,90E+11",Pinhalzinho,19,M,1,parda,1,1,nao_respondeu,0,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,Espanhol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sim,0
2,"1,90E+11",São Paulo,17,F,1,parda,1,2,privada,0,15567761.0,Belém,1,1,1,1,576.2,641.1,634.3,731.8,Inglês,1.0,160.0,120.0,200.0,200.0,200.0,880.0,sim,4
3,"1,90E+11",São Bernardo do Campo,19,M,1,parda,1,2,publica,0,35904958.0,São Bernardo do Campo,1,1,1,1,402.2,424.8,509.6,393.2,Espanhol,1.0,80.0,160.0,120.0,100.0,100.0,560.0,nao,4
4,"1,90E+11",São Joaquim da Barra,37,M,2,branca,1,1,nao_respondeu,0,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,Espanhol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nao,0
